# Visualização de Resultados
## Largura de desgaste e Coeficiente de Atrito

Foram realizados 60 ensaios, sendo eles:
- 12 ensaios em PU puro comercial - Axson F160, sintetizado segundo as recomendações do fornecedor.
- 16 ensaios em PU com 1%(massa) de nanoplaquetas de grafite (NPGs)
- 16 ensaios em PU com 2%(massa) de NPGs
- 16 ensaios em PU com 4%(massa) de NPGs

Os ensaios foram realizados em dois níveis de frequência, três níveis de força normal e mais uma condição central, com o valor médio de frequência e força normal. Os níveis foram os seguintes:
- Força Normal:
  - Alto: 8,5N
  - Médio: 5N
  - Baixo: 2,5N
  - Central: 5,17N
- Frequência:
  - Alta: 4Hz
  - Baixa: 2Hz
  - Central: 3Hz

Os ensaios foram realizados em duplicata, exceto os ensaios em condição central que foram realizados em quadruplicata. O ensaio em condição central não foi realizado para o PU puro.

### Etapa 1: Importando módulos e abrindo os arquivos de dados

As medições de largura de desgaste foram realizadas com auxílio de microscópio óptico, e seus valores foram manualmente digitados em uma planilha Excel ``Resultados Largura de Desgaste.xlsx``. Os resultados de coeficiente de atrito foram obtidos em tribômetro, exportados para ``.csv`` no aplicativo Viewer e compiladas com um script em Python desenvolvido anteriormente, resultando no arquivo ``reciprocating_data_new.csv``.

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio #graph output

import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np

outputfile = r'C:\Users\Caio\OneDrive\Documentos\Python\Results\Graphs\Resultados Largura de Desgaste.html'

filename = r'C:\Users\Caio\OneDrive\Documentos\Python\Results\Excel\Resultados Largura de Desgaste.xlsx'

df = pd.read_excel(filename) 

filename = r'C:\Users\Caio\OneDrive\Documentos\Python\Results\CSV\Compilado\reciprocating_data_new.csv'

df2 = pd.read_csv(filename)

### Etapa 2: Padronizando as tabelas

O dataframe ``df`` reúne os valores de Largura de Desgaste, enquanto o dataframe ``df2`` reúne todos os valores médios (COF e Largura de desgaste)

In [2]:
df['Largura Média'] = df[['Largura 1', 'Largura 2', 'Largura 3']].mean(1)
df['Largura Desv. Pad.'] = df[['Largura 1', 'Largura 2', 'Largura 3']].std(1)
df['Ensaio - Geral'] = 12*df['EG']+df['Ensaio']
df.loc[df['EG']==4,'Ensaio - Geral'] = 8+12*(df['EG']-1)+df['Ensaio']
df.sort_values(by = ['EG', 'Força Normal', 'Frequência'], inplace = True)

mat = {'PU': 0,
       'PUEG1': 1,
       'PUEG2': 2,
       'PUEG4': 4
      }
for material in df2['Material'].unique():
    df2.loc[df2['Material']==material,'EG'] = mat[material]
    df2.loc[df2['Material']==material,'PU'] = 100-mat[material]

In [3]:
for m in df['EG'].unique():
    for f in df['Força Normal'].unique():
        for fr in df['Frequência'].unique():
            mask = (df['EG']==m)&(df['Força Normal']==f)&(df['Frequência']==fr)
            mask2 = (df2['EG']==m)&(df2['Força Normal (N)']==f)&(df2['Frequência (Hz)']==fr)
            try:
                df2.loc[mask2,'Ensaio - Geral'] = df.loc[mask,'Ensaio - Geral'].unique()
            except ValueError:
                pass

In [4]:
for ensaio in df2['Ensaio - Geral'].unique():
    df2.loc[df2['Ensaio - Geral']==ensaio,'Largura Média'] = df.loc[df['Ensaio - Geral']==ensaio,'Largura Média'].mean()

In [5]:
df2.columns = ['indice',
               'Material',
               'Força Normal',
               'Frequência',
               'Distância',
               'COF Médio',
               'COF Desv. Pad.',
               'EG',
               'PU',
               'Ensaio - Geral',
               'Largura Média',
              ]

### Etapa 3: Visão geral dos ensaios (ordem cronológica)

Os ensaios foram realizados em ordem aleatorizada de força normal e frequência. Os primeiros 12 ensaios foram PU Puro, depois PU+1%NPG e PU+2%NPG (12 pontos extremos), pontos centrais 1% e 2%NPG e, por fim, os 16 ensaios de PU+4%NPGs.

In [116]:
fig = go.Figure()

fig.add_trace(df[df['Força Normal']==2.5].plot(kind = 'scatter', x = 'Ensaio - Geral', y = 'Largura Média', color = 'EG').data[0])
fig.add_trace(df[df['Força Normal']==5].plot(kind = 'scatter', x = 'Ensaio - Geral', y = 'Largura Média', color = 'EG').data[0])
fig.add_trace(df[df['Força Normal']==8].plot(kind = 'scatter', x = 'Ensaio - Geral', y = 'Largura Média', color = 'EG').data[0])


fig.data[0].error_y= dict(array = 1.96*df[df['Força Normal']==2.5]['Largura Desv. Pad.'],
                          type = 'data',
                          visible = True,
                          thickness = 1)
fig.data[1].error_y= dict(array = 1.96*df[df['Força Normal']==5]['Largura Desv. Pad.'],
                          type = 'data',
                          visible = True,
                          thickness = 1)
fig.data[2].error_y= dict(array = 1.96*df[df['Força Normal']==8]['Largura Desv. Pad.'],
                          type = 'data',
                          visible = True,
                          thickness = 1)

fig.data[0].marker.symbol = 'circle'
fig.data[1].marker.symbol = 'square'
fig.data[2].marker.symbol = 'diamond'

fig.data[0].name = '2,5N'
fig.data[1].name = '5N'
fig.data[2].name = '8N'

fig.data[0].showlegend = True
fig.data[1].showlegend = True
fig.data[2].showlegend = True

fig.update_layout(showlegend = True,
                  xaxis_title = 'Ensaio - Ordem',
                  yaxis_title = 'Largura de Desgaste (µm)',
                  legend = dict(
                      xanchor = 'left',
                      yanchor = 'bottom',
                      x = 0,
                      y = 1,
                      orientation = 'h',
                      title_text = 'Força Normal: '
                  ),
                  coloraxis = dict(colorscale = 'Bluered',
                                  colorbar_title = 'NPG (%)',
                                  ),
                  annotations = (dict(text = 'Os pontos sem barra de erro representam uma única medição',
                                      visible = True,
                                      x=1,
                                      y=1,
                                      showarrow = False,
                                      xref = 'paper',
                                      yref = 'paper',
                                      xanchor = 'right',
                                      yanchor = 'bottom'
                                    ),)
                 )

In [117]:
fig = go.Figure()

f2 = px.box(df[df['Frequência']==2],
             y='Largura Média',
             x = 'EG',
             color = 'Força Normal',
            )

f4 = px.box(df[df['Frequência']==4],
             y='Largura Média',
             x = 'EG',
             color = 'Força Normal',
            )

for data in f2.data:
    data.visible = True
    fig.add_trace(data)
for data in f4.data:
    data.visible = False
    fig.add_trace(data)

fig.update_layout(xaxis_title = 'Fração de Nanoplaquetas de Grafite',
                  yaxis_title = 'Largura de Desgaste (µm)',
                  xaxis_ticksuffix = '%',
                  yaxis_tickmode = 'linear',
                  yaxis_dtick = 50,
                  yaxis_tick0 = 200,
                  legend = dict(orientation = 'h',
                                y = 1.05,
                                x = 1.0,
                                xanchor= 'right',
                                yanchor= 'bottom',
                                title = dict(text = 'Força Normal (N):')
                               )
                 )

updatemenus=[
       dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label='2 Hz',
                     method="update",
                     args=[{"visible": [True]*3+[False]*3},
                           ]),
                dict(label='4 Hz',
                     method="update",
                     args=[{"visible": [False]*3+[True]*3},
                           ]),

            ]),
        )
    ]
fig.update_layout(updatemenus=updatemenus)

In [118]:
fig = go.Figure()

fig.add_trace(df2[df2['Força Normal']==2.5].plot(kind = 'scatter', y = 'COF Médio', x = 'Ensaio - Geral', color = 'EG').data[0])
fig.add_trace(df2[df2['Força Normal']==5].plot(kind = 'scatter', y = 'COF Médio', x = 'Ensaio - Geral', color = 'EG').data[0])
fig.add_trace(df2[df2['Força Normal']==5.17].plot(kind = 'scatter', y = 'COF Médio', x = 'Ensaio - Geral', color = 'EG').data[0])
fig.add_trace(df2[df2['Força Normal']==8].plot(kind = 'scatter', y = 'COF Médio', x = 'Ensaio - Geral', color = 'EG').data[0])

fig.data[0].error_y= dict(array = 1.96*df2[df2['Força Normal']==2.5]['COF Desv. Pad.'],
                          type = 'data',
                          visible = True,
                          thickness = 1)
fig.data[1].error_y= dict(array = 1.96*df2[df2['Força Normal']==5]['COF Desv. Pad.'],
                          type = 'data',
                          visible = True,
                          thickness = 1)
fig.data[2].error_y= dict(array = 1.96*df2[df2['Força Normal']==5.17]['COF Desv. Pad.'],
                          type = 'data',
                          visible = True,
                          thickness = 1)
fig.data[3].error_y= dict(array = 1.96*df2[df2['Força Normal']==8]['COF Desv. Pad.'],
                          type = 'data',
                          visible = True,
                          thickness = 1)


fig.data[0].marker.symbol = 'circle'
fig.data[1].marker.symbol = 'square'
fig.data[2].marker.symbol = 'diamond'
fig.data[3].marker.symbol = 'triangle-up'

fig.data[0].name = '2,5N'
fig.data[1].name = '5N'
fig.data[2].name = '5,17N'
fig.data[3].name = '8N'

fig.data[0].showlegend = True
fig.data[1].showlegend = True
fig.data[2].showlegend = True
fig.data[3].showlegend = True

fig.update_layout(showlegend = True,
                  yaxis_title = 'COF Médio (-)',
                  xaxis_title = 'Ensaio - Ordem',
                  legend = dict(
                      xanchor = 'left',
                      yanchor = 'bottom',
                      x = 0,
                      y = 1,
                      orientation = 'h',
                      title_text = 'Força Normal: '
                  ),
                  coloraxis = dict(colorscale = 'Bluered',
                                  colorbar_title = 'NPG (%)',
                                  ),
                  
                 )

In [119]:
titles = {0: 'PU Puro',
          1: 'PU + 1% NPGs',
          2: 'PU + 2% NPGs',
          4: 'PU + 4% NPGs'
            }

fig = go.Figure()
for eg in df['EG'].unique():
    mask = (df['EG']==eg)
    fig.add_trace(go.Scatter3d(
                                x=df[mask]['Força Normal'],
                                y=df[mask]['Frequência'],
                                z=df[mask]['Largura Média'],
                                mode="markers",
                                legendgroup=titles[eg],
                                name = titles[eg],
                                ))
scene = dict(xaxis_title = 'Força Normal (N)',
             yaxis_title = 'Frequência (Hz)',
             zaxis_title = 'Largura de desgaste (µm)'
            )
fig.update_layout(scene = scene,
                  title_text = 'Dados de Largura de Desgaste',
                  autosize = True,
                  #width=1000,
                  #height=1000,
                 )
fig.show()

# with open(outputfile, 'a') as f:
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [121]:
titles = {0: 'PU Puro',
          1: 'PU + 1% NPGs',
          2: 'PU + 2% NPGs',
          4: 'PU + 4% NPGs'
            }

fig = go.Figure()
for eg in df2['EG'].unique():
    mask = (df2['EG']==eg)
    fig.add_trace(go.Scatter3d(
                                x=df2[mask]['Força Normal'],
                                y=df2[mask]['Frequência'],
                                z=df2[mask]['COF Médio'],
                                mode="markers",
                                legendgroup=titles[eg],
                                name = titles[eg],
                                ))
scene = dict(xaxis_title = 'Força Normal (N)',
             yaxis_title = 'Frequência (Hz)',
             zaxis_title = 'COF (-)'
            )
fig.update_layout(scene = scene,
                  title_text = 'Dados de Coeficiente de Atrito',
                  autosize = True,
                  #width=1000,
                  #height=1000,
                 )
fig.show()

# with open(outputfile, 'a') as f:
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

### Etapa 4: Cálculo da regressão linear e análise de variância (ANOVA)

In [11]:
ndf = df

In [12]:
ndf = (df.assign(Intercept=1) #inclui uma coluna de 'intercepto' (fatorial) para o cálculo do coeficiente b0
#Aqui devem ser incluídos os parâmetros do modelo, até o máximo de N < número de níveis usados no experimento
          #.assign(**{'Força²' : ndf['Força Normal']**2}) #inclui parâmetro quadrático de força normal no modelo
          #.assign(**{'EG Percentage²' : ndf['EG Percentage']**2}) #inclui um parâmetro exponencial de porcentagem de grafite expandido
          #.assign(**{'Frequência * EG Percentage' : ndf['Frequência']*ndf['EG Percentage']}) #10
          .assign(**{'Força * EG' : ndf['Força Normal']*ndf['EG']}) #11
          #.assign(**{'Força * Frequência' : ndf['Força Normal']*ndf['Frequência']}) #12
          #.assign(**{'Força * Frequência * EG Percentage': ndf['Força Normal']*ndf['Frequência']*ndf['EG Percentage']}) #13
          #.assign(**{'Nome do novo parâmetro': definição do novo parâmetro}) #14
          )

In [13]:
ndf2 = df2.sort_values(by = ['EG','Força Normal','Frequência'])

In [14]:
k = 1/3
ndf2 = (df2.assign(Intercept=1) #inclui uma coluna de 'intercepto' (fatorial) para o cálculo do coeficiente b0
#Aqui devem ser incluídos os parâmetros do modelo, até o máximo de N < número de níveis usados no experimento
          .assign(**{'Força²' : ndf2['Força Normal']**2}) #inclui parâmetro quadrático de força normal no modelo
          
          .assign(**{'EG²' : ndf2['EG']**2}) #inclui um parâmetro exponencial de porcentagem de grafite expandido
          .assign(**{'Frequência²' : ndf2['Frequência']**2})
          .assign(**{'Frequência * EG' : ndf2['Frequência']*ndf2['EG']}) #10
          .assign(**{'Força * EG' : ndf2['Força Normal']*ndf2['EG']}) #11
          .assign(**{'Força * Frequência' : ndf2['Força Normal']*ndf2['Frequência']}) #12
          .assign(**{'Força * Frequência * EG': ndf2['Força Normal']*ndf2['Frequência']*ndf2['EG']}) #13
          .assign(**{'Força_Archard': ndf2['Força Normal']**(k)})
          #.assign(**{'Nome do novo parâmetro': definição do novo parâmetro}) #14
          )

In [15]:
ndf2 = (ndf2.assign(**{'Força_a * EG' : ndf2['Força_Archard']*ndf2['EG']})
            .assign(**{'Força_a * Frequência' : ndf2['Força_Archard']*ndf2['Frequência']})
            .assign(**{'Força_a * Frequência * EG': ndf2['Força_Archard']*ndf2['Frequência']*ndf2['EG']})
       )

#### Estimativa do desvio padrão

Posteriormente deverá ser calculada uma estimativa melhor do desvio padrão. Por enquanto é apenas a média dos desvios de cada ensaio.

In [16]:
s = ndf['Largura Desv. Pad.'].mean()

In [17]:
s2 = df2['COF Desv. Pad.'].mean()

In [18]:
P = ndf.columns.to_list()

In [19]:
parametros = [P[13], #intercept
              P[8], # força
              P[9], # freq
              P[2], # egp
              #P[14], #força*eg
              ]
resposta = P[10]

In [20]:
P2 = ndf2.columns.to_list()

In [21]:
parametros2 = [P2[11], #intercept
               #P2[2], # força
               P2[3], # freq
               #P2[7], # egp
               #P2[11], #força²
               P2[19], #força**[(1/4) a (1/3)]
               P2[13], #eg²
               P2[14], #freq²
               P2[15], #freq*eg
               #P2[15], #força*eg
               #P2[16], #força*freq
               #P2[17], #força*freq*eg
               P2[20],
               P2[21],
               P2[22],
               #P2[23]
              ]
resposta2 = P2[5]

#### Cálculo matricial para obtenção dos coeficientes 'b' e suas variâncias

Equação 5.12 de Barros Neto (2001):

$$ b = (X^TX)^{-1}X^Ty $$

Equação 5.30 de Barros Neto (2001):

$$ V(b) = (X^TX)^{-1}\sigma^2 $$

Onde **X** é a matriz de parâmetros que, no caso da largura de desgaste, por exemplo, é:

$\begin{bmatrix}
\textbf{$1$} & \textbf{$F_N$} & \textbf{$f$} & \textbf{$NPG_{\%}$} \\
1 & 2,5 & 2 & 0 \\
1 & 2,5 & 4 & 0 \\
1 & 5 & 2 & 0 \\
1 & 5 & 4 & 0 \\
1 & 8 & 2 & 0 \\
1 & 8 & 4 & 0 \\
1 & 2,5 & 2 & 1 \\
1 & 2,5 & 4 & 1 \\
1 & 5 & 2 & 1 \\
1 & 5 & 4 & 1 \\
1 & 8 & 2 & 1 \\
1 & 8 & 4 & 1 \\
\vdots & \vdots & \vdots & \vdots \\
1 & 5,17 & 3 & 4 \\
1 & 5,17 & 3 & 4 \\
1 & 5,17 & 3 & 4 \\
1 & 5,17 & 3 & 4 \\
1 & 8 & 2 & 4 \\
1 & 8 & 4 & 4 \\
\end{bmatrix}$

O vetor **y** é a coluna de respostas (largura de desgaste ou coeficiente de atrito), e $\sigma$ é o desvio padrão populacional, que foi estimado anteriormente nas variáveis ``s`` e ``s2``.

No caso do coeficiente de atrito, em vez de utilizar a força normal como parâmetro, está sendo usada a ${F_N}^{\frac{1}{3}}$, considerando a proposição de Archard que $COF \propto {F_N}^k$, com $\frac{1}{4}<=k<=\frac{1}{3}$. Além disso, estão sendo adicionados os parâmetros ${{NGP}_{\%}}^2$, $f^2$, ${{NGP}_{\%}}^2.f$,${F_N}^{\frac{1}{3}}.{{NGP}_{\%}}^2$, ${F_N}^{\frac{1}{3}}.f$ e ${F_N}^{\frac{1}{3}}.{{NGP}_{\%}}^2.f$.

O processo de obtenção do modelo é iterativo. Quando um coeficiente $b_i$ é menor que $1,96*V(b_i)^{\frac{1}{2}}$, é considerada a hipótese nula de que $b_i=0$ e, portanto, o parâmetro $i$ é retirado do modelo. 

In [22]:
X = ndf[parametros].to_numpy() #matriz de parâmetros X
Y = ndf[resposta].to_numpy().T #matriz de respostas Y

b = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),Y) #cálculo do vetor de parâmetros b 
Vb = (np.linalg.inv(np.dot(X.T,X)))*s**2 #variância/covariância dos parâmetros b, com base na variância calculada na primeira parte do programa
Vb = np.asmatrix(np.diag(Vb)).T #organizada como o vetor b (exclui covariancias)

In [23]:
X2 = ndf2[parametros2].to_numpy() #matriz de parâmetros X
Y2 = ndf2[resposta2].to_numpy().T #matriz de respostas Y

b2 = np.dot(np.dot(np.linalg.inv(np.dot(X2.T,X2)),X2.T),Y2) #cálculo do vetor de parâmetros b 
Vb2 = (np.linalg.inv(np.dot(X2.T,X2)))*s2**2 #variância/covariância dos parâmetros b, com base na variância calculada na primeira parte do programa
Vb2 = np.asmatrix(np.diag(Vb2)).T #organizada como o vetor b (exclui covariancias)

In [125]:
def anova(data,entradas,resp,coef,coef_var,matriz,nome):
    #cálculo das variáveis para a ANOVA
    #respostas calculadas com o modelo
    data = data.sort_values(by = ['EG','Força Normal','Frequência'])
    y_hat = 0
    for i in range(len(coef)):
        param = data[entradas]
        y_hat = y_hat + (coef.item(i)*param[param.columns[i]])

    #respostas médias para as repetiçoes de ensaios
    y_bar = []
    m = 0
    for eg in np.sort(data['EG'].unique()):
        for forca in np.sort(data['Força Normal'].unique()):
            for freq in np.sort(data['Frequência'].unique()):
                mask = (data['EG']==eg)&(data['Força Normal']==forca)&(data['Frequência']==freq)
                if data.loc[mask,resp].size>0:
                    m = m+1
                    for i in range(data.loc[mask,resp].size):
                        y_bar.append(data.loc[mask,resp].mean())


    p = matriz.shape[1] #número de parâmetros do modelo
    n = data[resp].count() #número total de observações

    SQR = ((y_hat-data[resp].mean())**2).sum() #soma quadrática da regressão
    GLR = p-1 #graus de liberdade da regressão
    MQR = SQR/GLR #média quadrática da regressão

    SQr = ((data[resp]-y_hat)**2).sum() #soma quadrática dos resíduos
    GLr = n-p #graus de liberdade dos resíduos
    MQr = SQr/GLr #média quadrática dos resíduos

    SQep = ((data[resp]-y_bar)**2).sum() #soma quadrática devido ao erro puro
    GLep = n-m #graus de liberdade devido ao erro puro
    MQep = SQep/GLep #média quadrática devido ao erro puro

    SQfaj = SQr - SQep #soma quadrática devido a falta de ajuste
    GLfaj = m-p #graus de liberdade devido a falta de ajuste
    MQfaj = SQfaj/GLfaj #média quadrática devido a falta de ajuste

    SQT = SQR+SQr #soma quadrática total
    GLT = n-1 #graus de liberdade total

    pve = SQR/SQT #porcentagem de variação explicada

    pmve = (SQT-SQep)/SQT #porcentagem de variação máxima explicável

    #imprime o modelo e a ANOVA
    print('Modelo:\n')
    for value,error,parametro in zip(coef,coef_var,entradas):
        if parametro == 'Intercept':
            print('{} = '.format(nome)+'{:.4f}±{:.5f} +'.format(value.item(0),1.96*error.item(0)**(1/2)))
        else:
            print('({:.4f}±{:.5f}) {} +'.format(value.item(0),1.96*error.item(0)**(1/2),parametro))
                  
    print("""
    Fonte de variação     Soma Quadrática      No de G. L.     Média Quadrática
    Regressão             {:>7.6f}              {:>2}              {:<6.6f}
    Resíduos              {:>7.6f}              {:>2}              {:<6.6f}
    Falta de ajuste       {:>7.6f}              {:>2}              {:<6.6f}
    Erro puro             {:>7.6f}              {:>2}              {:<6.6f}
    Total                 {:>7.6f}              {:>2}               
    % de variação explicada: {:.4f}
    % máxima de variação explicável: {:.4f}
    """.format(SQR,GLR,MQR,
               SQr,GLr,MQr,
               SQfaj,GLfaj,MQfaj,
               SQep,GLep,MQep,
               SQT,GLT,
               pve,
               pmve))


#### Resultado da ANOVA no modelo de regressão linear

In [126]:
anova(ndf,parametros,resposta,b,Vb,X,'L')

Modelo:

L = 176.6655±8.59580 +
(32.1574±0.91813) Força Normal +
(-3.9467±2.10028) Frequência +
(17.3542±1.32707) EG +

    Fonte de variação     Soma Quadrática      No de G. L.     Média Quadrática
    Regressão             595567.097221               3              198522.365740
    Resíduos              27526.549869              109              252.537155
    Falta de ajuste       10721.946971              21              510.568903
    Erro puro             16804.602898              88              190.961397
    Total                 623093.647090              112               
    % de variação explicada: 0.9558
    % máxima de variação explicável: 0.9730
    


In [127]:
anova(ndf2,parametros2,resposta2,b2,Vb2,X2,'COF')

Modelo:

COF = -0.2367±0.16269 +
(0.3755±0.09092) Frequência +
(0.0975±0.07207) Força_Archard +
(0.0097±0.00298) EG² +
(-0.0381±0.01336) Frequência² +
(-0.0320±0.00888) Frequência * EG +
(-0.0270±0.01033) Força_a * EG +
(-0.0795±0.02415) Força_a * Frequência +
(0.0153±0.00562) Força_a * Frequência * EG +

    Fonte de variação     Soma Quadrática      No de G. L.     Média Quadrática
    Regressão             0.137846               8              0.017231
    Resíduos              0.067513              51              0.001324
    Falta de ajuste       0.042658              18              0.002370
    Erro puro             0.024855              33              0.000753
    Total                 0.205359              59               
    % de variação explicada: 0.6712
    % máxima de variação explicável: 0.8790
    


### Etapa 6: Gráficos

Gráficos 3D da superfície e gráficos 2D de mapa de calor calculados através do modelo de regressão linear são plotados aqui. Um controle deslizante foi adicionado para mostrar as respostas para diferentes composições dos materiais. Ao final, um gráfico customizável foi adicionado, mas ele não roda direto no html.

In [27]:
Fo = np.linspace(2.5, 8.5, 10) #entradas de força de 2,5 a 8,0 N de 0,5 em 0,5N
Eg = np.linspace(0, 4.0, 10) #entradas de porcentagem de EG de 0 a 2 %, de 0,1 em 0,1%
Fr = np.linspace(2,4,10) #entradas de frequencia de 2 a 4 Hz, de 0,2 em 0,2

Ff,ff = np.meshgrid(Fo,Fr)

In [28]:
def answer2d(egp,freq,F,b):
    resposta = []
    for fr in freq:
        #parametros do modelo
        Alpha = np.array((1,
                       F,
                       fr,
                       egp,
                       ),)
        resposta.append(np.diag(b*Alpha).sum())
    return np.array(resposta)

In [29]:
def answer2d2(egp,freq,F,b2):
    resposta = []
    for fr in freq:
        P = np.array((1,
                      #F,
                      fr,
                      #egp,
                      F**(k),
                      egp**2,
                      fr**2,
                      fr*egp,
                      (F**(k))*egp,
                      (F**(k))*fr,
                      (F**(k))*fr*egp
                     ),)
        resposta.append(np.diag(b2*P).sum())
    return np.array(resposta)

In [30]:
fig = go.Figure()

for e_slider in np.arange(0,4.1,0.1): 
    fig.add_trace(go.Surface(z=answer2d(e_slider,Fr,Fo,b),
                              x=Ff,
                              y=ff,
                              opacity=0.6,
                              name='PU+{:.1f}%NPG'.format(e_slider),
                              coloraxis = 'coloraxis',
                              colorbar = dict(thickness=5),
                              visible = False
                              ))
fig.data[10].visible = True
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        label = '{:.1f}% NPG'.format(i/10),
        args=[{"visible": [False] * len(fig.data)}, 
              
              {"title": "Previsão do modelo de desgaste do PU + {:.1f}% NPG".format(i/10)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Content: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders,
                  coloraxis = dict(colorscale = 'hot',
                                   colorbar = dict(thickness = 5)
                                  ),
                  title='Previsão do modelo de desgaste do PU + 1.0% NPG',
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene = dict(xaxis_title = 'Força Normal (N)',
                               yaxis_title = 'Frequência (Hz)',
                               zaxis_title = 'Largura de Desgaste (µm)'),
                  showlegend=True,
                  legend=dict(yanchor="top",
                              y=0.95,
                              xanchor="left",
                              x=0.05),
                  scene_zaxis=dict(range=[220, 500]),
                  scene_xaxis=dict(showspikes = False),
                  scene_yaxis=dict(showspikes = False),
                  autosize = True,
                  #width=1000,
                  #height=1000,
                  )
# with open(outputfile, 'a') as f:
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [31]:
fig = go.Figure()

for e_slider in np.arange(0,4.1,0.1): 
    fig.add_trace(go.Contour(z=answer2d(e_slider,Fr,Fo,b),
                             x=Fo,
                             y=Fr,
                             opacity=0.85,
                             name='PU+{:.1f}%NPG'.format(e_slider),
                             coloraxis = 'coloraxis',
                             contours=dict(coloring = 'heatmap',
                                           end = 500,
                                           start = 200,
                                           showlabels=True,
                                          ),
                             colorbar = dict(thickness=5),
                             visible = False,
                            ))
fig.data[10].visible = True
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        label = '{:.1f}% NPG'.format(i/10),
        args=[{"visible": [False] * len(fig.data)}, 
              
              {"title": "Previsão do modelo de desgaste do PU + {:.1f}% NPG".format(i/10)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Content: "},
    pad={"t": 50},
    len = 1.2,
    steps=steps
)]

fig.update_layout(sliders=sliders,
                  coloraxis = dict(colorscale = 'Portland',
                                   colorbar = dict(
                                       title_text = 'Largura de Desgaste (µm)',
                                       thickness = 10,
                                       title_side = 'right'
                                   )
                                  ),
                  title='Previsão do modelo de desgaste do PU + 1.0% NPG',
                  margin=dict(l=65, r=50, b=65, t=90),
                  xaxis_title = 'Força Normal (N)',
                  yaxis_title = 'Frequência (Hz)',
                  showlegend=True,
                  autosize = False,
                  width=500,
                  height=500,
                  )
# with open(outputfile, 'a') as f:
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [32]:
fig = go.Figure()

for e_slider in np.arange(0,4.1,0.1): 
    fig.add_trace(go.Surface(z=answer2d2(e_slider,Fr,Fo,b2),
                              x=Ff,
                              y=ff,
                              opacity=0.6,
                              name='PU+{:.1f}%EG'.format(e_slider),
                              coloraxis = 'coloraxis',
                              colorbar = dict(thickness=5),
                              visible = False
                              ))
fig.data[10].visible = True
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        label = '{:.1f}% EG'.format(i/10),
        args=[{"visible": [False] * len(fig.data)}, 
              
              {"title": "Previsão do modelo de atrito do PU + {:.1f}% NPG".format(i/10)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Content: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders,
                  coloraxis = dict(colorscale = 'hot',
                                   colorbar = dict(thickness = 5)
                                  ),
                  title='Previsão do modelo de atrito do PU + 1.0% NPG',
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene = dict(xaxis_title = 'Força Normal (N)',
                               yaxis_title = 'Frequência (Hz)',
                               zaxis_title = 'Coeficiente de Atrito (-)'),
                  showlegend=True,
                  legend=dict(yanchor="top",
                              y=0.95,
                              xanchor="left",
                              x=0.05),
                  scene_zaxis=dict(range=[0.1, 0.42]),
                  scene_xaxis=dict(showspikes = False),
                  scene_yaxis=dict(showspikes = False),
                  autosize = True,
                  #width=1000,
                  #height=1000,
                  )
# with open(outputfile, 'a') as f:
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

#### Gráfico invertendo os eixos: x: F_N y: NGP% z: COF, slider: freq

In [33]:
fig = go.Figure()

Ff,ee = np.meshgrid(Fo,Eg)

def answer2_freq_F(egp,freq,F,b2):
    resposta = []
    for eg in egp:
        P = np.array((1,
                      #F,
                      freq,
                      #eg,
                      F**(k),
                      eg**2,
                      freq**2,
                      freq*eg,
                      (F**(k))*eg,
                      (F**(k))*freq,
                      (F**(k))*freq*eg
                     ),)
        resposta.append(np.diag(b2*P).sum())
    return np.array(resposta)

for f_slider in np.arange(2,4.1,0.1): 
    fig.add_trace(go.Surface(z=answer2_freq_F(Eg,f_slider,Fo,b2),
                              x=Ff,
                              y=ee,
                              opacity=0.6,
                              name='{}Hz'.format(f_slider),
                              coloraxis = 'coloraxis',
                              colorbar = dict(thickness=5),
                              visible = False
                              ))
fig.data[10].visible = True
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        label = '{}Hz'.format(2+i/10),
        args=[{"visible": [False] * len(fig.data)}, 
              
              {"title": "Previsão do modelo de atrito com {}Hz".format(2+i/10)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Content: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders,
                  coloraxis = dict(colorscale = 'hot',
                                   colorbar = dict(thickness = 5)
                                  ),
                  title='Previsão do modelo de atrito com 2Hz',
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene = dict(xaxis_title = 'Força Normal (N)',
                               yaxis_title = 'NPG (%)',
                               zaxis_title = 'Coeficiente de Atrito (-)'),
                  showlegend=True,
                  legend=dict(yanchor="top",
                              y=0.95,
                              xanchor="left",
                              x=0.05),
                  scene_zaxis=dict(range=[0.1, 0.42]),
                  scene_xaxis=dict(showspikes = False),
                  scene_yaxis=dict(showspikes = False),
                  autosize = True,
                  #width=1000,
                  #height=1000,
                  )
# with open(outputfile, 'a') as f:
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [34]:
fig = go.Figure()

Ff,ff = np.meshgrid(Fo,Fr)

for e_slider in np.arange(0,4.1,0.1): 
    fig.add_trace(go.Contour(z=answer2d2(e_slider,Fr,Fo,b2),
                             x=Fo,
                             y=Fr,
                             opacity=0.85,
                             name='PU+{:.1f}%NPG'.format(e_slider),
                             coloraxis = 'coloraxis',
                             contours=dict(coloring = 'heatmap',
                                           end = 0.45,
                                           start = 0.12,
                                           showlabels=True),
                             colorbar = dict(thickness=5),
                             visible = False,
                            ))
fig.data[10].visible = True
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        label = '{:.1f}% NPG'.format(i/10),
        args=[{"visible": [False] * len(fig.data)}, 
              
              {"title": "Previsão do modelo de atrito do PU + {:.1f}% NPG".format(i/10)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Content: "},
    pad={"t": 50},
    len = 1.2,
    steps=steps
)]

fig.update_layout(sliders=sliders,
                  coloraxis = dict(colorscale = 'Portland',
                                   colorbar = dict(
                                       title_text = 'Coeficiente de atrito (-)',
                                       thickness = 10,
                                       title_side = 'right',
#                                        tickmode = 'array',
#                                        tickvals = [0.15,0.2,0.25,0.3,0.35,0.4,0.45],
#                                        ticktext = ['0.150','0.200','0.250','0.300','0.350','0.400','0.450'],
                                   ),
                                  ),
                  title='Previsão do modelo de atrito do PU + 1.0% NPG',
                  margin=dict(l=65, r=50, b=65, t=90),
                  xaxis_title = 'Força Normal (N)',
                  yaxis_title = 'Frequência (Hz)',
                  showlegend=True,
                  
                  autosize = False,
                  width=500,
                  height=550,
                  )
# with open(outputfile, 'a') as f:
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [65]:
#ndf2.to_csv(r'C:\Users\Caiot\OneDrive\Documentos\Python\Results\CSV\Compilado\reciprocating_anova.csv')

In [109]:
def plot_Heatmap(NPG,var,color='Portland',opacity=0.85,w=500,h=500):
    fig = go.Figure()
    if var == 'COF':
        fig.add_trace(go.Contour(z=answer2d2(NPG,Fr,Fo,b2),
                                 x=Fo,
                                 y=Fr,
                                 opacity=opacity,
                                 name='PU+{:.1f}%NPG'.format(NPG),
                                 coloraxis = 'coloraxis',
                                 colorbar = dict(thickness=5),
                                 visible = True,
                                 contours=dict(coloring = 'heatmap',
                                               end = 0.45,
                                               start = 0.12,
                                               showlabels=True),
                                )
                     )
        fig.update_layout(title='Previsão do modelo de atrito do PU + {}% NPG'.format(NPG),
                          scene_zaxis_title = 'Coeficiente de Atrito (-)',
                         )
    elif var == 'L':
        fig.add_trace(go.Contour(z=answer2d(NPG,Fr,Fo,b),
                                 x=Fo,
                                 y=Fr,
                                 opacity=opacity,
                                 name='PU+{:.1f}%NPG'.format(NPG),
                                 coloraxis = 'coloraxis',
                                 contours=dict(coloring = 'heatmap',
                                               end = 500,
                                               start = 200,
                                               showlabels=True,
                                              ),
                                 colorbar = dict(thickness=5),
                                 visible = True
                                )
                     )
        
        fig.update_layout(title='Previsão do modelo de desgaste do PU + {}% NPG'.format(NPG),
                          scene_zaxis_title = 'Largura de Desgaste (µm)',
                         )
    fig.update_layout(
                  coloraxis = dict(colorscale = color,
                                   colorbar = dict(
                                       title_text = 'Coeficiente de atrito (-)',
                                       thickness = 10,
                                       title_side = 'right',
                                   ),
                                  ),
                      title='Previsão do modelo de atrito do PU + {}% NPG'.format(NPG),
                  margin=dict(l=65, r=50, b=65, t=90),
                  xaxis_title = 'Força Normal (N)',
                  yaxis_title = 'Frequência (Hz)',
                  showlegend=True,
                  autosize = False,
                  width=w,
                  height=h,
                  )
    fig.show()

In [128]:
def plot_3Dsurface(NPG,var,color='hot',opacity=0.6,w=650,h=650):
    fig = go.Figure()
    if var == 'COF':
        fig.add_trace(go.Surface(z=answer2d2(NPG,Fr,Fo,b2),
                                      x=Ff,
                                      y=ff,
                                      opacity=opacity,
                                      name='PU+{:.1f}%NPG'.format(NPG),
                                      coloraxis = 'coloraxis',
                                      colorbar = dict(thickness=5),
                                      visible = True
                                      ))
        fig.update_layout(title='Previsão do modelo de atrito do PU + {}% NPG'.format(NPG),
                          scene_zaxis_title = 'Coeficiente de Atrito (-)',
                          scene_zaxis_range = [0.1, 0.42],
                         )
    elif var == 'L':
        fig.add_trace(go.Surface(z=answer2d(NPG,Fr,Fo,b),
                                      x=Ff,
                                      y=ff,
                                      opacity=opacity,
                                      name='PU+{:.1f}%NPG'.format(NPG),
                                      coloraxis = 'coloraxis',
                                      colorbar = dict(thickness=5),
                                      visible = True
                                      ))

        fig.update_layout(title='Previsão do modelo de desgaste do PU + {}% NPG'.format(NPG),
                          scene_zaxis_title = 'Largura de Desgaste (µm)',
                          scene_zaxis_range = [250, 500],
                         )
    fig.update_layout(coloraxis = dict(colorscale = color,
                                           colorbar = dict(thickness = 5)
                                          ),
                          
                          margin=dict(l=65, r=50, b=65, t=90),
                          scene = dict(xaxis_title = 'Força Normal (N)',
                                       yaxis_title = 'Frequência (Hz)',
                                      ),
                          showlegend=True,
                          legend=dict(yanchor="top",
                                      y=0.95,
                                      xanchor="left",
                                      x=0.05),
                          
                          scene_xaxis=dict(showspikes = False),
                          scene_yaxis=dict(showspikes = False),
                          autosize = False,
                          width=w,
                          height=h,
                          )
    fig.show()

In [129]:
from ipywidgets import interact, widgets

In [130]:
interact(plot_3Dsurface,
         NPG=(0,4,0.1),
         var=['COF','L'],
         color=px.colors.named_colorscales(),
         opacity = (0,1,0.05),
         w = (100,2000),
         h = (100,2000),
        )
        

interactive(children=(FloatSlider(value=2.0, description='NPG', max=4.0), Dropdown(description='var', options=…

<function __main__.plot_3Dsurface(NPG, var, color='hot', opacity=0.6, w=650, h=650)>

In [113]:
interact(plot_Heatmap,
         NPG=(0,4,0.1),
         var=['COF','L'],
         color=px.colors.named_colorscales(),
         opacity = (0,1,0.05),
         w = (100,2000),
         h = (100,2000),
        )
        

interactive(children=(FloatSlider(value=2.0, description='NPG', max=4.0), Dropdown(description='var', options=…

<function __main__.plot_Heatmap(NPG, var, color='Portland', opacity=0.85, w=500, h=500)>